## Import Libraries:

In [1]:
from jupyter_dash import JupyterDash
import dash
from dash import html 
from dash import dcc
from dash.dependencies import Input, Output, State

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

import numpy as np 
import pandas as pd

## Loading Dataset:

In [2]:
df = pd.read_csv('london_crime_by_lsoa.csv')
#df = df.iloc[:5000, :]

In [3]:
df.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13490604 entries, 0 to 13490603
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   lsoa_code       object
 1   borough         object
 2   major_category  object
 3   minor_category  object
 4   value           int64 
 5   year            int64 
 6   month           int64 
dtypes: int64(3), object(4)
memory usage: 720.5+ MB


In [5]:
df.isnull().sum().sum()

0

In [6]:
df.duplicated().sum()

0

In [7]:
crimes = df.major_category.unique()
geo_df = pd.read_csv('london-city-lat-lng.csv')

In [8]:
app = JupyterDash(external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"])

In [9]:
crime_type = sorted(df["major_category"].unique())
# DropDownBox: For chossing crime
DropDownCity = dcc.Dropdown(
    id = 'drop_crime',
    options = [{'label': crime_type[i], 'value': crime_type[i]} for i in range(len(crime_type))],
    multi = False,
    value = "Burglary",
style= {'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':20,'padding': '15px', 'backgroundColor': 'rgb(212, 218,220)'})


In [10]:
app.layout = html.Div(
    
    children=[
    
        html.Div(className='row',# Define the row element
                 children=[
        
               
            # bans
    
            html.Div([html.H2("London Crime Analysis", style= {'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':45})
                     ], className = 'four columns', style = {'textAlign' : 'center', 'padding': '5px' ,'backgroundColor': 'rgb(212, 218,220)'}),
        
            html.Div([
                html.H2(id='MaxYear', style= {'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':25}),
                html.H4('YEAR', style={'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':15}),
                ], className = 'two columns', style = {'textAlign' : 'center', 'backgroundColor': 'rgb(212, 218,220)'}),

            html.Div([
                html.H2(id='MaxMonth', style= {'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':25}),
                html.H4('MONTH', style={'textAlign': 'center', 'color': 'black', 'fontFamily':'Alkes Regular Italic', 'fontSize':15})
                ], className = 'two columns', style = {'textAlign' : 'center', 'backgroundColor': 'rgb(212, 218,220)'}),

            html.Div([
                html.H2(id='MaxCity', style= {'textAlign': 'center', 'color': 'crimson', 'fontFamily':'Alkes Regular Italic', 'fontSize':25}),
                html.H4('CITY', style={'textAlign': 'center', 'color': 'crimson', 'fontFamily':'Alkes Regular Italic', 'fontSize':15})
                ], className = 'two columns', style = {'textAlign' : 'center', 'backgroundColor': 'rgb(212, 218,220)'}),

            
            html.Div(
                DropDownCity
                , className = 'two columns')
                 ])
         
    ,
    
        html.Div(className='row',  # Define the row element
        children=[
            html.Div(
                dcc.Graph(id='bar_graph'), className='six columns div-user-controls'),
            html.Div(
                dcc.Graph(id="map_graph"), className='six columns div-user-controls')
            ])
     ,   
        html.Div(className='row',  # Define the row element
        children=[
            html.Div(
                dcc.Graph(id="year_graph"), className='six columns div-user-controls'),
            html.Div(
                dcc.Graph(id="month_graph"), className='six columns div-user-controls')
        ])
    
    ])



In [11]:
@app.callback(
    Output("bar_graph", "figure"),
    Output("map_graph", "figure"), 
    Output("year_graph", "figure"), 
    Output("month_graph", "figure"),
    
    Output(component_id="MaxYear", component_property='children'),
    Output(component_id="MaxMonth", component_property='children'),
    Output(component_id="MaxCity", component_property='children'),

    Input(component_id="drop_crime", component_property='value'))

def display_choropleth(crime):
    
    df1 = df[df["major_category"] == crime]
    grouped_df = df1.groupby(["year", "month", "borough"])["value"].sum().sort_values().to_frame().reset_index()
    
    year = grouped_df.iloc[-1, 0]
    month = grouped_df.iloc[-1, 1]
    city = grouped_df.iloc[-1, 2]
    
    # BarChart Figure 
    bar_df = grouped_df[(grouped_df["year"] == year) & (grouped_df["month"] == month)]
    colors = ['lightslategray'] * bar_df.shape[0]
    colors[-1] = 'crimson'
    bar = go.Figure(data=[go.Bar(x=bar_df["borough"], y=bar_df["value"], marker_color=colors)])
    bar.update_layout(template="none")
    bar.update_layout(template="none", title_text="How do the sum of crimes vary by City in {0}".format(year))
    
    # Map Graph
    c1 = df1.groupby(['borough'])['value'].sum().to_frame().reset_index()

    merged_data= pd.merge(geo_df, c1, on="borough")    
    fig1 = px.scatter_mapbox(merged_data, lat=merged_data.lat, lon=merged_data.lng, color="value", zoom=5,size ='value', hover_name="borough",
                        mapbox_style='carto-positron', color_continuous_scale=px.colors.sequential.Reds)
    fig1.update_layout(template="none")
    fig1.update_layout(template="none", title_text="Sum of crimes vary by City")
    
    # Year Graph
    c2 = df.groupby(['major_category', 'year'])['value'].sum().to_frame().reset_index()    
    fig2 = px.line(c2, x='year', y='value', color='major_category', markers = 'major_category')
    fig2.update_layout(template="none")
    fig2.update_layout(template="none", title_text="How do the sum of crimes vary by year")
    
    # Month Graph
    c3 = df[df['major_category']== crime].groupby(['month', 'year'])['value'].sum().to_frame().reset_index()  
    fig3 = px.line(c3, x='month', y='value', color='year', markers = 'year')
    fig3.update_layout(template="none")
    fig3.update_layout(template="none", title_text="How do the sum of crimes vary by month")

    return bar, fig1, fig2, fig3, year, month, city


In [12]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [13]:
df.shape

(13490604, 7)